--- Day 10: Knot Hash ---
You come across some programs that are trying to implement a software emulation of a hash based on knot-tying. The hash these programs are implementing isn't very strong, but you decide to help them anyway. You make a mental note to remind the Elves later not to invent their own cryptographic functions.

This hash function simulates tying a knot in a circle of string with 256 marks on it. Based on the input to be hashed, the function repeatedly selects a span of string, brings the ends together, and gives the span a half-twist to reverse the order of the marks within it. After doing this many times, the order of the marks is used to build the resulting hash.

  4--5   pinch   4  5           4   1
 /    \  5,0,1  / \/ \  twist  / \ / \
3      0  -->  3      0  -->  3   X   0
 \    /         \ /\ /         \ / \ /
  2--1           2  1           2   5
To achieve this, begin with a list of numbers from 0 to 255, a current position which begins at 0 (the first element in the list), a skip size (which starts at 0), and a sequence of lengths (your puzzle input). Then, for each length:

Reverse the order of that length of elements in the list, starting with the element at the current position.
Move the current position forward by that length plus the skip size.
Increase the skip size by one.
The list is circular; if the current position and the length try to reverse elements beyond the end of the list, the operation reverses using as many extra elements as it needs from the front of the list. If the current position moves past the end of the list, it wraps around to the front. Lengths larger than the size of the list are invalid.

Here's an example using a smaller list:

Suppose we instead only had a circular list containing five elements, 0, 1, 2, 3, 4, and were given input lengths of 3, 4, 1, 5.

The list begins as [0] 1 2 3 4 (where square brackets indicate the current position).
The first length, 3, selects ([0] 1 2) 3 4 (where parentheses indicate the sublist to be reversed).
After reversing that section (0 1 2 into 2 1 0), we get ([2] 1 0) 3 4.
Then, the current position moves forward by the length, 3, plus the skip size, 0: 2 1 0 [3] 4. Finally, the skip size increases to 1.
The second length, 4, selects a section which wraps: 2 1) 0 ([3] 4.
The sublist 3 4 2 1 is reversed to form 1 2 4 3: 4 3) 0 ([1] 2.
The current position moves forward by the length plus the skip size, a total of 5, causing it not to move because it wraps around: 4 3 0 [1] 2. The skip size increases to 2.
The third length, 1, selects a sublist of a single element, and so reversing it has no effect.
The current position moves forward by the length (1) plus the skip size (2): 4 [3] 0 1 2. The skip size increases to 3.
The fourth length, 5, selects every element starting with the second: 4) ([3] 0 1 2. Reversing this sublist (3 0 1 2 4 into 4 2 1 0 3) produces: 3) ([4] 2 1 0.
Finally, the current position moves forward by 8: 3 4 2 1 [0]. The skip size increases to 4.
In this example, the first two numbers in the list end up being 3 and 4; to check the process, you can multiply them together to produce 12.

However, you should instead use the standard list size of 256 (with values 0 to 255) and the sequence of lengths in your puzzle input. Once this process is complete, what is the result of multiplying the first two numbers in the list?

In [196]:
class Ele(object):
    def __init__(self):
        self.parent = None
        self.child = None
        self.first = False
        self.value = None
        
    def __repr__(self):
        if self.first:
            return "<{}>".format(self.value)            
        return "{}".format(self.value)
        
class Slist(object):
    def __init__(self, values):
        self.eles = []
        self.step = 0
        
        last = None
        first = True
        for v in values:
            e = Ele()
            e.value = v
            e.parent = last
            e.first = first
            first = False
            if last:
                last.child = e
            last = e
            self.eles.append(e)
            
        if self.first and e:
            e.child = self.first
            self.parent = e
        
            
    def __repr__(self):
        return "<{}>".format(self.eles)
    
    @property
    def first(self):
        for ele in self.eles:
            if ele.first:
                return ele
            
    def first_index(self):
        for i, ele in enumerate(self.eles):
            if ele.first:
                return i
            
    @property
    def score(self):
        return self.first.value * self.first.child.value
            
    @property
    def length(self):
        return len(self.eles)

    
    def mirror(self, bunch):
        b = bunch-1
        for a in range(int(bunch/2)):
            hold = self.eles[a].value
            self.eles[a].value = self.eles[b].value
            self.eles[b].value = hold
            b -= 1
    
    
    def rotate(self, move):
        stop =  move % self.length
        newend = self.eles[:stop]
        newstart = self.eles[stop:]
        self.eles = newstart + newend
    
            
    def process(self, instructions):
        for inst in instructions:
            inst = int(inst)
            self.mirror(inst)
            move = inst+self.step
            self.rotate(move)
            self.step += 1
            
    def dense_hash(self):
        ret = []
        
        first = self.first_index()
        self.rotate(first)
        
        for i in range(0, 255, 16):
            values = [x.value for x in self.eles[i:i+16]]
            s = 0
            for x in values:
                s ^= x
            ret.append(s)
        return ret
                    
    def hexrep(self):
        hexes = [hex(x) for x in self.dense_hash()]
        ret = ""
        for h in hexes:
            rep = h.split('x')[1]
            rep = "{:0>2}".format(rep)
            ret += rep
        return ret
    
    def repeat(self, count, instructions):
        for i in range(count):
            self.process(instructions)
    
    @staticmethod
    def toascii(s):
        salt = [17, 31, 73, 47, 23]
        values = [ord(x) for x in s]
        values.extend(salt)        
        return values 


In [98]:
actual = [0, 1, 2, 3, 4]
actual = [x for x in range(256)]
actual[0], actual[-1]

(0, 255)

In [99]:
inst = [3,4,1,5]
inst = "199,0,255,136,174,254,227,16,51,85,1,2,22,17,7,192"
inst = Slist.toascii(inst)

print(inst)
# inst = [49,44,50,44,51,17,31,73,47,23]

[49, 57, 57, 44, 48, 44, 50, 53, 53, 44, 49, 51, 54, 44, 49, 55, 52, 44, 50, 53, 52, 44, 50, 50, 55, 44, 49, 54, 44, 53, 49, 44, 56, 53, 44, 49, 44, 50, 44, 50, 50, 44, 49, 55, 44, 55, 44, 49, 57, 50]


In [835]:

slst = Slist(actual)
print(slst)
print("\n<<<<first:{}, step: {}>>>>\n".format(slst.first, slst.step))

for i in range(64):
    slst.process(inst)
    print(slst)
    print("<<<<first:{}, step: {}>>>>\n".format(slst.first, slst.step))
    
print(slst)
len(slst.hexrep()), slst.hexrep()


<[<0>, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 2

(32, 'a9d0e68649d0174c8756a59ba21d4dc6')

#####See Day 14######

In [204]:
inst = 'flqrgnkx-1'
inp = Slist.toascii(inst)
inp

[102, 108, 113, 114, 103, 110, 107, 120, 45, 49, 17, 31, 73, 47, 23]

In [205]:
s = Slist(actual)

In [206]:
s.repeat(64, inp)

In [207]:
s.dense_hash()

[85, 234, 179, 196, 251, 254, 222, 22, 220, 236, 44, 102, 221, 162, 100, 100]

In [208]:
hx = s.hexrep()

In [209]:
def tobin(hsh):
    ret = bin(int(hsh, 16))
    ret = ret.split('b')
    ret = '{:0>4}'.format(ret[1])
    return ret

In [231]:
tots = []
count = 0
for i in range(128):
    key = "flqrgnkx-" + str(i)
    key = Slist.toascii(key)
    cal = Slist(actual)
    cal.repeat(64, key)
    hsh = cal.hexrep()
    binbits = [tobin(x) for x in hsh]
    bits = ''.join(binbits)
    count += sum([int(x) for x in bits])
    tots.append(bits)
    

In [232]:
tots

['11010100111101110110101111011100101111111000001110001111100001000001011011001100111110101000101111000110110100011111100111100110',
 '01010101111010101011001111000100111110111111111011011110000101101101110011101100001011000110011011011101101000100110010001100100',
 '00001010110111110001001111111010010000001110100011101010100000010101001101110110011101110110101011110011101101111011001000110001',
 '10101101001111011010001010001100110101111011100011111011100110010111010000101100000011100110001101100111001011001010111101100010',
 '01101000001011111110010010001100010101011000011101101010101010101010000100011101111100100110001101001111100101101101001100011010',
 '11001001111101011111111111111100010001100100101001010110010111111001111010010100111100110010001110101000101010101010100101001011',
 '01000100110000000001101001110010011000100110110011101110000011100011101111000000111010001001100010001101000100110111010001100011',
 '1101011001110010001101100000100000110000110001100011100101001001010